<a href="https://colab.research.google.com/github/khalidjasir/ui-km-paper/blob/khalid-v2/KM_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Install necessary library

In [ ]:
%pip install numpy==1.24.4
%pip install scipy==1.10.1
%pip install scikit-learn==1.2.2
%pip install gensim==4.3.2
%pip install pandas==1.5.3
%pip install fasttext
%pip install tqdm
%pip install ipywidgets
%pip install swifter
%pip install emoji
%pip install indonlp
%pip install openpyxl
%pip install telethon
%pip install pytz
%pip install python-dotenv

  Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2025.3.1 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
dask-expr 1.1.21 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.1 requi

  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.10.1 which is incompatible.


## Detect environment

In [ ]:
import os
import sys
import subprocess

def detect_environment():
  try:
    import google.colab
    return 'colab'
  except ImportError:
    pass

  if 'CONDA_PREFIX' in os.environ or os.path.exists(os.path.join(sys.prefix, 'conda-meta')):
    return 'conda'

  return 'local'

env = detect_environment()

## Handle Key

In [ ]:
api_id = ''
api_hash = ''

if env == 'colab':
  from google.colab import userdata
  api_id = userdata.get('api_id')
  api_hash = userdata.get('api_hash')
elif env == 'conda':
  from dotenv import load_dotenv
  load_dotenv()
  api_id = os.getenv('api_id')
  api_hash = os.getenv('api_hash')
else:
  print('Unable to detect suitable environment!')

## Retrieve from telegram

In [ ]:
import csv
import time
from datetime import datetime
from telethon import TelegramClient
from pytz import timezone

session_name = 'my_session'
channel_input = 'https://t.me/diskusipajak'
wib_timezone = timezone('Asia/Jakarta')
start_date = wib_timezone.localize(datetime(2015, 1, 1))
end_date = wib_timezone.localize(datetime(2025, 4, 30))
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_filename = f'telegram_messages_{timestamp}.csv'
batch_size = 10000

async def main():
  async with TelegramClient(session_name, api_id, api_hash) as client:
    channel = await client.get_entity(channel_input)

    buffer = []
    total_count = 0

    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
      writer = csv.writer(file, quoting=csv.QUOTE_ALL)
      writer.writerow([
        'id',
        'date',
        'text',
        'sender_id',
        'chat_id',
        'reply_to_msg_id',
        'views',
        'forwards',
        'buttons',
        'raw_text',
        'message_link'
      ])

    async for msg in client.iter_messages(channel, offset_date=start_date, reverse=True):
      if msg.date > end_date:
        break

      row = [
        msg.id,
        msg.date.astimezone(wib_timezone).strftime('%a %b %d %H:%M:%S %z %Y') if msg.date else "",
        msg.text.replace('\n', ' ').strip() if msg.text else "",
        msg.sender_id if msg.sender_id else '',
        getattr(msg, 'chat_id', getattr(msg.to_id, 'channel_id', '')),
        msg.reply_to_msg_id if msg.reply_to_msg_id else '',
        msg.views if msg.views is not None else '',
        msg.forwards if msg.forwards is not None else '',
        len(msg.buttons) if msg.buttons else 0,
        msg.raw_text.replace('\n', ' ').strip() if msg.raw_text else '',
        f'https://t.me/{channel.username}/{msg.id}'
      ]

      buffer.append(row)
      total_count += 1

      if total_count % batch_size == 0:
        with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
          writer = csv.writer(file, quoting=csv.QUOTE_ALL)
          writer.writerows(buffer)

        buffer.clear()
        print(f'Written {total_count} messages. Sleeping for 10 seconds...')
        time.sleep(10)

    if buffer:
      with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_ALL)
        writer.writerows(buffer)
      print(f'Final batch written. Total messages: {total_count}')

    print(f'Done! Messages saved to {csv_filename}.')

await main()

Written 10000 messages. Sleeping for 10 seconds...
Written 20000 messages. Sleeping for 10 seconds...
Written 30000 messages. Sleeping for 10 seconds...
Written 40000 messages. Sleeping for 10 seconds...
Written 50000 messages. Sleeping for 10 seconds...
Written 60000 messages. Sleeping for 10 seconds...
Written 70000 messages. Sleeping for 10 seconds...
Written 80000 messages. Sleeping for 10 seconds...
Written 90000 messages. Sleeping for 10 seconds...
Written 100000 messages. Sleeping for 10 seconds...
Written 110000 messages. Sleeping for 10 seconds...
Written 120000 messages. Sleeping for 10 seconds...
Written 130000 messages. Sleeping for 10 seconds...
Written 140000 messages. Sleeping for 10 seconds...
Written 150000 messages. Sleeping for 10 seconds...
Written 160000 messages. Sleeping for 10 seconds...
Written 170000 messages. Sleeping for 10 seconds...
Written 180000 messages. Sleeping for 10 seconds...
Written 190000 messages. Sleeping for 10 seconds...
Written 200000 messag

## Move file for later processing

In [ ]:
if env == 'colab':
  file_path = '/content/drive/MyDrive/telegram-data/'
  !mkdir -p $file_path
  !mv $csv_filename $file_path
  print(f'Successfully moved {csv_filename} to {file_path}')
elif env == 'conda':
  file_path = './data/'
  !mkdir -p $file_path
  !mv $csv_filename $file_path
  print(f'Successfully moved {csv_filename} to {file_path}')
else:
  print('Unable to detect suitable environment! No file moved')

Successfully moved telegram_messages_20250519_042802.csv to /content/drive/MyDrive/telegram-data/


## Retrieve stored telegram messages

In [ ]:
import pandas as pd

filename = 'telegram_messages_20250519_042802.csv'

try:
  if env == 'colab':
    file_path = f'/content/drive/MyDrive/telegram-data/{filename}'
    messages_df = df = pd.read_csv(file_path)
    print(f'Successfully read from {file_path} to dataframe')
  elif env == 'conda':
    file_path = f'./data/{filename}'
    messages_df = df = pd.read_csv(file_path)
    print(f'Successfully read from {file_path} to dataframe')
  else:
    print('Unable to detect suitable environment! Nothing loaded to dataframe')
except FileNotFoundError:
  messages_df = pd.DataFrame()
  print(f'File not found')
except Exception as e:
  messages_df = pd.DataFrame()
  print(f'Error reading file: {e}')

display(messages_df)

Successfully read from /content/drive/MyDrive/telegram-data/telegram_messages_20250519_042802.csv to dataframe


,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,message_link
0,1,Thu Dec 31 13:10:16 +0700 2015,NaN,NaN,-1001014841061,NaN,NaN,NaN,0,NaN,https://t.me/diskusipajak/1
1,69,Thu Dec 31 14:45:57 +0700 2015,Grup membahas seputar permasalahan aplikasi ya...,8.580854e+07,-1001014841061,NaN,NaN,NaN,0,Grup membahas seputar permasalahan aplikasi ya...,https://t.me/diskusipajak/69
2,71,Thu Dec 31 14:54:33 +0700 2015,iya pak,8.472804e+07,-1001014841061,NaN,NaN,NaN,0,iya pak,https://t.me/diskusipajak/71
3,74,Thu Dec 31 14:56:16 +0700 2015,"pak untuk ebiling, itu per januari 2016 semua ...",8.472804e+07,-1001014841061,NaN,NaN,NaN,0,"pak untuk ebiling, itu per januari 2016 semua ...",https://t.me/diskusipajak/74
4,77,Thu Dec 31 14:59:04 +0700 2015,makasih pak,8.472804e+07,-1001014841061,NaN,NaN,NaN,0,makasih pak,https://t.me/diskusipajak/77
...,...,...,...,...,...,...,...,...,...,...,...
331201,533652,Tue Apr 29 19:37:32 +0700 2025,punya ku kalo dicek data2nya udah oke semua. g...,1.275726e+09,-1001014841061,533650.0,NaN,NaN,0,punya ku kalo dicek data2nya udah oke semua. g...,https://t.me/diskusipajak/533652
331202,533653,Tue Apr 29 19:42:58 +0700 2025,STP muncul karena telat bayar ppn masa Januari...,6.648129e+09,-1001014841061,NaN,NaN,NaN,0,STP muncul karena telat bayar ppn masa Januari...,https://t.me/diskusipajak/533653
331203,533654,Tue Apr 29 20:00:47 +0700 2025,"hehe, tadi pagi kami bahas ini disebelah",8.580854e+07,-1001014841061,533653.0,NaN,NaN,0,"hehe, tadi pagi kami bahas ini disebelah",https://t.me/diskusipajak/533654
331204,533655,Tue Apr 29 20:01:52 +0700 2025,Ijin tanya kawan Coretax masih lemot GK ya? L...,1.284664e+09,-1001014841061,NaN,NaN,NaN,0,Ijin tanya kawan Coretax masih lemot GK ya? L...,https://t.me/diskusipajak/533655


## Merging reply with original message

In [ ]:
import pandas as pd

messages_df.dropna(subset=['text', 'raw_text'], inplace=True)

row_lookup = {
  (row['chat_id'], row['id']): row.to_dict() for _, row in messages_df.iterrows()
}

def collect_reply_chain(chat_id, msg_id, visited=None):
  if visited is None:
    visited = set()

  key = (chat_id, msg_id)
  if key in visited:
    return []
  visited.add(key)

  current_row = row_lookup.get(key)
  if current_row is None:
    return []

  reply_to_id = current_row.get('reply_to_msg_id')
  if pd.isna(reply_to_id):
    return [current_row]

  parent_chain = collect_reply_chain(chat_id, reply_to_id, visited)
  return parent_chain + [current_row]

def format_chain_sorted(chain):
  chain_sorted = sorted(chain, key=lambda x: x['date'])
  return ' RT '.join(str(row.get('text') or '') for row in chain_sorted)

messages_df['merged_text'] = messages_df.apply(
  lambda row: format_chain_sorted(collect_reply_chain(row['chat_id'], row['id'])), axis=1
)

## Pre-processing functions

In [ ]:
import unicodedata
import emoji
import re
import string
import indoNLP.preprocessing

def basic_clean(text):
  # Normalize Unicode (remove fancy fonts, underlines)
  text = unicodedata.normalize('NFKD', text)
  text = ''.join(c for c in text if not unicodedata.combining(c))

  # Translate emojis
  text = emoji.demojize(text, delimiters=(' ', ' '), language='id')

  # Remove HTML
  text = indoNLP.preprocessing.remove_html(text)

  # Remove URL
  text = indoNLP.preprocessing.remove_url(text)

  # Remove usernames
  text = re.sub(r'@\w+', '', text)

  # Remove RT
  text = re.sub(r'\brt\b', '', text, flags=re.IGNORECASE)

  # Remove hashtag symbol but keep the word
  text = re.sub(r'#', '', text)

  # Remove extra whitespace
  text = re.sub(r'\s+', ' ', text).strip()

  return text

def advanced_clean(text):
  # Remove word elongation
  text = indoNLP.preprocessing.replace_word_elongation(text)

  # Replace slang
  text = indoNLP.preprocessing.replace_slang(text)

  # Lowercase
  text = text.lower()

  # Remove punctuation
  text = text.translate(str.maketrans('', '', string.punctuation))

  # Remove digits
  text = re.sub(r'\d+', '', text)

  # Remove stopwords
  text = indoNLP.preprocessing.remove_stopwords(text)

  # Remove extra whitespace again (just in case)
  text = re.sub(r'\s+', ' ', text).strip()

  return text

## Run pre-processing

In [ ]:
import swifter

messages_df.dropna(subset=['text', 'raw_text', 'merged_text'], inplace=True)
messages_df['basic_clean'] = messages_df['merged_text'].swifter.apply(basic_clean)
messages_df['advanced_clean'] = messages_df['basic_clean'].swifter.apply(advanced_clean)
messages_df.dropna(subset=['basic_clean', 'advanced_clean'], inplace=True)

display(messages_df)

Pandas Apply:   0%|          | 0/305356 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/305356 [00:00<?, ?it/s]

,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,message_link,merged_text,basic_clean,advanced_clean
1,69,Thu Dec 31 14:45:57 +0700 2015,Grup membahas seputar permasalahan aplikasi ya...,8.580854e+07,-1001014841061,NaN,NaN,NaN,0,Grup membahas seputar permasalahan aplikasi ya...,https://t.me/diskusipajak/69,Grup membahas seputar permasalahan aplikasi ya...,Grup membahas seputar permasalahan aplikasi ya...,grup membahas seputar permasalahan aplikasi di...
2,71,Thu Dec 31 14:54:33 +0700 2015,iya pak,8.472804e+07,-1001014841061,NaN,NaN,NaN,0,iya pak,https://t.me/diskusipajak/71,iya pak,iya pak,iya
3,74,Thu Dec 31 14:56:16 +0700 2015,"pak untuk ebiling, itu per januari 2016 semua ...",8.472804e+07,-1001014841061,NaN,NaN,NaN,0,"pak untuk ebiling, itu per januari 2016 semua ...",https://t.me/diskusipajak/74,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp
4,77,Thu Dec 31 14:59:04 +0700 2015,makasih pak,8.472804e+07,-1001014841061,NaN,NaN,NaN,0,makasih pak,https://t.me/diskusipajak/77,makasih pak,makasih pak,terima kasih
5,78,Thu Dec 31 15:05:40 +0700 2015,shortcut nya hanya via *141*500# telkomsel aja ya,8.580854e+07,-1001014841061,NaN,NaN,NaN,0,shortcut nya hanya via *141*500# telkomsel aja ya,https://t.me/diskusipajak/78,shortcut nya hanya via *141*500# telkomsel aja ya,shortcut nya hanya via *141*500 telkomsel aja ya,shortcut nya via telkomsel ya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331201,533652,Tue Apr 29 19:37:32 +0700 2025,punya ku kalo dicek data2nya udah oke semua. g...,1.275726e+09,-1001014841061,533650.0,NaN,NaN,0,punya ku kalo dicek data2nya udah oke semua. g...,https://t.me/diskusipajak/533652,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...
331202,533653,Tue Apr 29 19:42:58 +0700 2025,STP muncul karena telat bayar ppn masa Januari...,6.648129e+09,-1001014841061,NaN,NaN,NaN,0,STP muncul karena telat bayar ppn masa Januari...,https://t.me/diskusipajak/533653,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...
331203,533654,Tue Apr 29 20:00:47 +0700 2025,"hehe, tadi pagi kami bahas ini disebelah",8.580854e+07,-1001014841061,533653.0,NaN,NaN,0,"hehe, tadi pagi kami bahas ini disebelah",https://t.me/diskusipajak/533654,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...
331204,533655,Tue Apr 29 20:01:52 +0700 2025,Ijin tanya kawan Coretax masih lemot GK ya? L...,1.284664e+09,-1001014841061,NaN,NaN,NaN,0,Ijin tanya kawan Coretax masih lemot GK ya? L...,https://t.me/diskusipajak/533655,Ijin tanya kawan Coretax masih lemot GK ya? L...,Ijin tanya kawan Coretax masih lemot GK ya? La...,ijin kawan coretax lemot ya buka


## Filter out text that is shorter than 5 words

In [ ]:
telegram_df = messages_df[messages_df['advanced_clean'].str.split().str.len() > 5].copy()

display(telegram_df)

,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,message_link,merged_text,basic_clean,advanced_clean
1,69,Thu Dec 31 14:45:57 +0700 2015,Grup membahas seputar permasalahan aplikasi ya...,8.580854e+07,-1001014841061,NaN,NaN,NaN,0,Grup membahas seputar permasalahan aplikasi ya...,https://t.me/diskusipajak/69,Grup membahas seputar permasalahan aplikasi ya...,Grup membahas seputar permasalahan aplikasi ya...,grup membahas seputar permasalahan aplikasi di...
3,74,Thu Dec 31 14:56:16 +0700 2015,"pak untuk ebiling, itu per januari 2016 semua ...",8.472804e+07,-1001014841061,NaN,NaN,NaN,0,"pak untuk ebiling, itu per januari 2016 semua ...",https://t.me/diskusipajak/74,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp
9,83,Thu Dec 31 15:25:18 +0700 2015,db nya di backup dulu,8.580854e+07,-1001014841061,80.0,NaN,NaN,0,db nya di backup dulu,https://t.me/diskusipajak/83,pak mau tanya cara creat database pph4(2) baga...,pak mau tanya cara creat database pph4(2) baga...,creat database pph ya db nya backup
17,91,Thu Dec 31 15:27:13 +0700 2015,ya ada npwpx... nah ganti npwpx memang ndak bs...,1.206142e+08,-1001014841061,NaN,NaN,NaN,0,ya ada npwpx... nah ganti npwpx memang ndak bs...,https://t.me/diskusipajak/91,ya ada npwpx... nah ganti npwpx memang ndak bs...,ya ada npwpx... nah ganti npwpx memang ndak bs...,ya npwpx ganti npwpx indak ya
35,113,Thu Dec 31 18:38:48 +0700 2015,masih bisa untuk 3 bank bumn dan kantor pos sa...,6.223018e+07,-1001014841061,74.0,NaN,NaN,0,masih bisa untuk 3 bank bumn dan kantor pos sa...,https://t.me/diskusipajak/113,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp bank bu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331201,533652,Tue Apr 29 19:37:32 +0700 2025,punya ku kalo dicek data2nya udah oke semua. g...,1.275726e+09,-1001014841061,533650.0,NaN,NaN,0,punya ku kalo dicek data2nya udah oke semua. g...,https://t.me/diskusipajak/533652,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...
331202,533653,Tue Apr 29 19:42:58 +0700 2025,STP muncul karena telat bayar ppn masa Januari...,6.648129e+09,-1001014841061,NaN,NaN,NaN,0,STP muncul karena telat bayar ppn masa Januari...,https://t.me/diskusipajak/533653,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...
331203,533654,Tue Apr 29 20:00:47 +0700 2025,"hehe, tadi pagi kami bahas ini disebelah",8.580854e+07,-1001014841061,533653.0,NaN,NaN,0,"hehe, tadi pagi kami bahas ini disebelah",https://t.me/diskusipajak/533654,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...
331204,533655,Tue Apr 29 20:01:52 +0700 2025,Ijin tanya kawan Coretax masih lemot GK ya? L...,1.284664e+09,-1001014841061,NaN,NaN,NaN,0,Ijin tanya kawan Coretax masih lemot GK ya? L...,https://t.me/diskusipajak/533655,Ijin tanya kawan Coretax masih lemot GK ya? L...,Ijin tanya kawan Coretax masih lemot GK ya? La...,ijin kawan coretax lemot ya buka


## Retrieving language model

In [ ]:
import os
import fasttext
import urllib.request

filename = 'lid.176.bin'
model_url = 'https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin'
model_path = ''

try:
  if env == 'colab':
    model_dir = '/content/drive/MyDrive/models'
  elif env == 'conda':
    model_dir = './models'
  else:
    raise EnvironmentError('Unable to detect suitable environment!')

  os.makedirs(model_dir, exist_ok=True)
  model_path = os.path.join(model_dir, filename)

  if not os.path.exists(model_path):
    print('Model not found locally. Downloading...')
    urllib.request.urlretrieve(model_url, model_path)
    print('Download completed!')

  print(f'Loading model from {model_path}...')
  model = fasttext.load_model(model_path)
  print('Model loaded successfully!')

except Exception as e:
  print(f'Error loading FastText model: {e}')

Loading model from /content/drive/MyDrive/models/lid.176.bin...
Model loaded successfully!


## Identify language

In [ ]:
texts = telegram_df['advanced_clean'].tolist()

predictions = model.predict(texts)

telegram_df['lang_detected'] = [label[0].replace('__label__', '') for label in predictions[0]]
telegram_df['lang_confidence'] = [float(score[0]) for score in predictions[1]]

lang_stats = telegram_df.groupby('lang_detected')['lang_confidence'].agg(
  count='count',
  min='min',
  q25=lambda x: x.quantile(0.25),
  mean='mean',
  median='median',
  q90=lambda x: x.quantile(0.9),
  max='max'
).reset_index().sort_values(by='count', ascending=False)

display(lang_stats)

,lang_detected,count,min,q25,mean,median,q90,max
34,id,169817,0.061187,0.428736,0.547046,0.553219,0.762665,0.993517
56,ms,11537,0.056299,0.304658,0.397730,0.392303,0.566405,0.934640
17,en,9204,0.057642,0.219940,0.327831,0.288599,0.542167,0.998555
84,sw,2706,0.060828,0.238129,0.351898,0.323929,0.570028,0.950363
15,de,770,0.084488,0.198911,0.288113,0.268254,0.448509,0.787191
...,...,...,...,...,...,...,...,...
50,lmo,1,0.223466,0.223466,0.223466,0.223466,0.223466,0.223466
45,ko,1,0.259508,0.259508,0.259508,0.259508,0.259508,0.259508
43,km,1,0.206750,0.206750,0.206750,0.206750,0.206750,0.206750
41,jbo,1,0.196097,0.196097,0.196097,0.196097,0.196097,0.196097


## Filter text with Bahasa Indonesia

In [ ]:
filtered_df = telegram_df[
  (telegram_df['lang_detected'] == 'id')
].copy()

display(filtered_df)

,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,message_link,merged_text,basic_clean,advanced_clean,lang_detected,lang_confidence
1,69,Thu Dec 31 14:45:57 +0700 2015,Grup membahas seputar permasalahan aplikasi ya...,8.580854e+07,-1001014841061,NaN,NaN,NaN,0,Grup membahas seputar permasalahan aplikasi ya...,https://t.me/diskusipajak/69,Grup membahas seputar permasalahan aplikasi ya...,Grup membahas seputar permasalahan aplikasi ya...,grup membahas seputar permasalahan aplikasi di...,id,0.838247
3,74,Thu Dec 31 14:56:16 +0700 2015,"pak untuk ebiling, itu per januari 2016 semua ...",8.472804e+07,-1001014841061,NaN,NaN,NaN,0,"pak untuk ebiling, itu per januari 2016 semua ...",https://t.me/diskusipajak/74,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp,id,0.424206
37,115,Thu Dec 31 18:39:08 +0700 2015,Makasih pak,8.472804e+07,-1001014841061,113.0,NaN,NaN,0,Makasih pak,https://t.me/diskusipajak/115,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp bank bu...,id,0.497953
42,120,Thu Dec 31 18:42:08 +0700 2015,untuk menggunakan aplikasi lapor pajak online ...,1.206142e+08,-1001014841061,NaN,NaN,NaN,0,untuk menggunakan aplikasi lapor pajak online ...,https://t.me/diskusipajak/120,untuk menggunakan aplikasi lapor pajak online ...,untuk menggunakan aplikasi lapor pajak online ...,aplikasi lapor pajak online mesti bayar daftar,id,0.618863
45,123,Thu Dec 31 18:43:22 +0700 2015,Saya dengar2 Pajak mau mengeluarkan laporan pa...,2.853580e+06,-1001014841061,NaN,NaN,NaN,0,Saya dengar2 Pajak mau mengeluarkan laporan pa...,https://t.me/diskusipajak/123,Saya dengar2 Pajak mau mengeluarkan laporan pa...,Saya dengar2 Pajak mau mengeluarkan laporan pa...,dengar pajak mengeluarkan laporan pajak online,id,0.546661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331201,533652,Tue Apr 29 19:37:32 +0700 2025,punya ku kalo dicek data2nya udah oke semua. g...,1.275726e+09,-1001014841061,533650.0,NaN,NaN,0,punya ku kalo dicek data2nya udah oke semua. g...,https://t.me/diskusipajak/533652,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...,id,0.605423
331202,533653,Tue Apr 29 19:42:58 +0700 2025,STP muncul karena telat bayar ppn masa Januari...,6.648129e+09,-1001014841061,NaN,NaN,NaN,0,STP muncul karena telat bayar ppn masa Januari...,https://t.me/diskusipajak/533653,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...,id,0.771047
331203,533654,Tue Apr 29 20:00:47 +0700 2025,"hehe, tadi pagi kami bahas ini disebelah",8.580854e+07,-1001014841061,533653.0,NaN,NaN,0,"hehe, tadi pagi kami bahas ini disebelah",https://t.me/diskusipajak/533654,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...,id,0.766360
331204,533655,Tue Apr 29 20:01:52 +0700 2025,Ijin tanya kawan Coretax masih lemot GK ya? L...,1.284664e+09,-1001014841061,NaN,NaN,NaN,0,Ijin tanya kawan Coretax masih lemot GK ya? L...,https://t.me/diskusipajak/533655,Ijin tanya kawan Coretax masih lemot GK ya? L...,Ijin tanya kawan Coretax masih lemot GK ya? La...,ijin kawan coretax lemot ya buka,id,0.428418


## Find similarity with KNN

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.8)
tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_df['advanced_clean'])

nn = NearestNeighbors(metric='cosine', n_neighbors=10)
nn.fit(tfidf_matrix)

distances, indices = nn.kneighbors(tfidf_matrix)

threshold = 0.95
to_remove = set()

for i, (dists, neighbors) in enumerate(zip(distances, indices)):
  for dist, idx in zip(dists[1:], neighbors[1:]):
    sim = 1 - dist
    if sim >= threshold:
      to_remove.add(max(i, idx))

unique_df = filtered_df.drop(filtered_df.index[list(to_remove)]).reset_index(drop=True)

display(unique_df)

,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,message_link,merged_text,basic_clean,advanced_clean,lang_detected,lang_confidence
0,69,Thu Dec 31 14:45:57 +0700 2015,Grup membahas seputar permasalahan aplikasi ya...,8.580854e+07,-1001014841061,NaN,NaN,NaN,0,Grup membahas seputar permasalahan aplikasi ya...,https://t.me/diskusipajak/69,Grup membahas seputar permasalahan aplikasi ya...,Grup membahas seputar permasalahan aplikasi ya...,grup membahas seputar permasalahan aplikasi di...,id,0.838247
1,74,Thu Dec 31 14:56:16 +0700 2015,"pak untuk ebiling, itu per januari 2016 semua ...",8.472804e+07,-1001014841061,NaN,NaN,NaN,0,"pak untuk ebiling, itu per januari 2016 semua ...",https://t.me/diskusipajak/74,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp,id,0.424206
2,115,Thu Dec 31 18:39:08 +0700 2015,Makasih pak,8.472804e+07,-1001014841061,113.0,NaN,NaN,0,Makasih pak,https://t.me/diskusipajak/115,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp bank bu...,id,0.497953
3,120,Thu Dec 31 18:42:08 +0700 2015,untuk menggunakan aplikasi lapor pajak online ...,1.206142e+08,-1001014841061,NaN,NaN,NaN,0,untuk menggunakan aplikasi lapor pajak online ...,https://t.me/diskusipajak/120,untuk menggunakan aplikasi lapor pajak online ...,untuk menggunakan aplikasi lapor pajak online ...,aplikasi lapor pajak online mesti bayar daftar,id,0.618863
4,123,Thu Dec 31 18:43:22 +0700 2015,Saya dengar2 Pajak mau mengeluarkan laporan pa...,2.853580e+06,-1001014841061,NaN,NaN,NaN,0,Saya dengar2 Pajak mau mengeluarkan laporan pa...,https://t.me/diskusipajak/123,Saya dengar2 Pajak mau mengeluarkan laporan pa...,Saya dengar2 Pajak mau mengeluarkan laporan pa...,dengar pajak mengeluarkan laporan pajak online,id,0.546661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122584,533650,Tue Apr 29 19:34:09 +0700 2025,Keterangannya NPWP pemotong tidak valid tapi b...,7.234777e+09,-1001014841061,NaN,NaN,NaN,0,Keterangannya NPWP pemotong tidak valid tapi b...,https://t.me/diskusipajak/533650,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...,id,0.648648
122585,533652,Tue Apr 29 19:37:32 +0700 2025,punya ku kalo dicek data2nya udah oke semua. g...,1.275726e+09,-1001014841061,533650.0,NaN,NaN,0,punya ku kalo dicek data2nya udah oke semua. g...,https://t.me/diskusipajak/533652,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...,id,0.605423
122586,533653,Tue Apr 29 19:42:58 +0700 2025,STP muncul karena telat bayar ppn masa Januari...,6.648129e+09,-1001014841061,NaN,NaN,NaN,0,STP muncul karena telat bayar ppn masa Januari...,https://t.me/diskusipajak/533653,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...,id,0.771047
122587,533654,Tue Apr 29 20:00:47 +0700 2025,"hehe, tadi pagi kami bahas ini disebelah",8.580854e+07,-1001014841061,533653.0,NaN,NaN,0,"hehe, tadi pagi kami bahas ini disebelah",https://t.me/diskusipajak/533654,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...,id,0.766360


## Store to local file

In [ ]:
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
excel_filename = f'filtered_telegram_messages_{timestamp}.xlsx'
unique_df.to_excel(excel_filename, index=False)

if env == 'colab':
  file_path = f'/content/drive/MyDrive/telegram-data/'
  !mkdir -p $file_path
  !mv $excel_filename $file_path
  print(f'Successfully moved {excel_filename} to {file_path}')
elif env == 'conda':
  file_path = f'./data/'
  !mkdir -p $file_path
  !mv $excel_filename $file_path
  print(f'Successfully moved {excel_filename} to {file_path}')
else:
  print('Unable to detect suitable environment!')

Successfully moved filtered_telegram_messages_20250519_140349.xlsx to /content/drive/MyDrive/telegram-data/


## Retrieve filtered messeges

In [ ]:
import pandas as pd

filename = excel_filename # 'filtered_telegram_messages_YYYYMMDD_hhmmss.xlsx'

try:
  if env == 'colab':
    file_path = f'/content/drive/MyDrive/telegram-data/{filename}'
    unique_df = df = pd.read_excel(file_path)
    print(f'Successfully read from {file_path} to dataframe')
  elif env == 'conda':
    file_path = f'./data/{filename}'
    unique_df = df = pd.read_excel(file_path)
    print(f'Successfully read from {file_path} to dataframe')
  else:
    print('Unable to detect suitable environment!')
except FileNotFoundError:
  unique_df = pd.DataFrame()
  print(f'File not found')
except Exception as e:
  unique_df = pd.DataFrame()
  print(f'Error reading file: {e}')

display(unique_df)

Successfully read from /content/drive/MyDrive/telegram-data/filtered_telegram_messages_20250519_140349.xlsx to dataframe


,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,message_link,merged_text,basic_clean,advanced_clean,lang_detected,lang_confidence
0,69,Thu Dec 31 14:45:57 +0700 2015,Grup membahas seputar permasalahan aplikasi ya...,8.580854e+07,-1001014841061,NaN,NaN,NaN,0,Grup membahas seputar permasalahan aplikasi ya...,https://t.me/diskusipajak/69,Grup membahas seputar permasalahan aplikasi ya...,Grup membahas seputar permasalahan aplikasi ya...,grup membahas seputar permasalahan aplikasi di...,id,0.838247
1,74,Thu Dec 31 14:56:16 +0700 2015,"pak untuk ebiling, itu per januari 2016 semua ...",8.472804e+07,-1001014841061,NaN,NaN,NaN,0,"pak untuk ebiling, itu per januari 2016 semua ...",https://t.me/diskusipajak/74,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp,id,0.424206
2,115,Thu Dec 31 18:39:08 +0700 2015,Makasih pak,8.472804e+07,-1001014841061,113.0,NaN,NaN,0,Makasih pak,https://t.me/diskusipajak/115,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp bank bu...,id,0.497953
3,120,Thu Dec 31 18:42:08 +0700 2015,untuk menggunakan aplikasi lapor pajak online ...,1.206142e+08,-1001014841061,NaN,NaN,NaN,0,untuk menggunakan aplikasi lapor pajak online ...,https://t.me/diskusipajak/120,untuk menggunakan aplikasi lapor pajak online ...,untuk menggunakan aplikasi lapor pajak online ...,aplikasi lapor pajak online mesti bayar daftar,id,0.618863
4,123,Thu Dec 31 18:43:22 +0700 2015,Saya dengar2 Pajak mau mengeluarkan laporan pa...,2.853580e+06,-1001014841061,NaN,NaN,NaN,0,Saya dengar2 Pajak mau mengeluarkan laporan pa...,https://t.me/diskusipajak/123,Saya dengar2 Pajak mau mengeluarkan laporan pa...,Saya dengar2 Pajak mau mengeluarkan laporan pa...,dengar pajak mengeluarkan laporan pajak online,id,0.546661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122584,533650,Tue Apr 29 19:34:09 +0700 2025,Keterangannya NPWP pemotong tidak valid tapi b...,7.234777e+09,-1001014841061,NaN,NaN,NaN,0,Keterangannya NPWP pemotong tidak valid tapi b...,https://t.me/diskusipajak/533650,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...,id,0.648648
122585,533652,Tue Apr 29 19:37:32 +0700 2025,punya ku kalo dicek data2nya udah oke semua. g...,1.275726e+09,-1001014841061,533650.0,NaN,NaN,0,punya ku kalo dicek data2nya udah oke semua. g...,https://t.me/diskusipajak/533652,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...,id,0.605423
122586,533653,Tue Apr 29 19:42:58 +0700 2025,STP muncul karena telat bayar ppn masa Januari...,6.648129e+09,-1001014841061,NaN,NaN,NaN,0,STP muncul karena telat bayar ppn masa Januari...,https://t.me/diskusipajak/533653,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...,id,0.771047
122587,533654,Tue Apr 29 20:00:47 +0700 2025,"hehe, tadi pagi kami bahas ini disebelah",8.580854e+07,-1001014841061,533653.0,NaN,NaN,0,"hehe, tadi pagi kami bahas ini disebelah",https://t.me/diskusipajak/533654,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...,id,0.766360


## Run LDA topic modeling

In [ ]:
from collections import defaultdict
from gensim import corpora, models
from gensim.models import CoherenceModel

unique_df['date'] = pd.to_datetime(unique_df['date'], format='%a %b %d %H:%M:%S %z %Y')
unique_df['year'] = unique_df['date'].dt.year
unique_df['month'] = unique_df['date'].dt.month

monthly_yearly_topics = defaultdict(dict)

for (month, year), group in unique_df.groupby(['month', 'year']):
  tokenized_docs = group['advanced_clean'].apply(str.split).tolist()

  if len(tokenized_docs) < 5:
    continue

  dictionary = corpora.Dictionary(tokenized_docs)
  dictionary.filter_extremes(no_below=3, no_above=0.85)
  corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]
  if not corpus:
    continue

  num_topics = 10

  lda_model = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    random_state=42,
    passes=10,
    alpha='auto',
    per_word_topics=True
  )

  topics = []
  for topic_id in range(num_topics):
    words_probs = lda_model.show_topic(topic_id, topn=7)
    words = [word for word, _ in words_probs]
    topics.append({
      'topic_id': topic_id,
      'top_words': words
    })

  coherence_model = CoherenceModel(
    model=lda_model,
    texts=tokenized_docs,
    dictionary=dictionary,
    coherence='c_v'
  )

  monthly_yearly_topics[month][year] = {
    'topics': topics,
    'coherence': coherence_model.get_coherence()
  }

month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

for target_month in range(1, 13):
  if target_month in monthly_yearly_topics:
    print(f'\n=== Comparing Topics for {month_names[target_month - 1]} ===\n')
    for year in sorted(monthly_yearly_topics[target_month]):
      print(f"\n--- {month_names[target_month - 1]} {year} (Coherence: {monthly_yearly_topics[target_month][year]['coherence']:.4f}) ---")
      for t in monthly_yearly_topics[target_month][year]['topics']:
        print(f"Topic {t['topic_id']}: {', '.join(t['top_words'])}")
  else:
    print(f'\n=== No data for {month_names[target_month - 1]} ===')


=== Comparing Topics for January ===


--- January 2016 (Coherence: 0.4341) ---
Topic 0: ya, kode, aplikasi, efaktur, ppn, potong, bukti
Topic 1: ppn, terima, kasih, ya, perusahaan, pph, npwp
Topic 2: pajak, faktur, kalo, bisnis, klik, dobel, pilih
Topic 3: faktur, nomor, pajak, silakan, coba, kp, orang
Topic 4: perpajakan, selamat, mas, indonesia, info, ya, dobel
Topic 5: pajak, bayar, kp, sih, kantor, pos, bank
Topic 6: kp, efaktur, kalo, espt, ya, folder, tinggal
Topic 7: wajib, efaktur, internet, langsung, pajak, efin, wp
Topic 8: ya, wp, aktivasi, efin, daftar, email, kalo
Topic 9: pajak, inisiatif, efin, strategis, transaksi, wajib, kalo

--- January 2017 (Coherence: 0.3044) ---
Topic 0: kalo, nya, pajak, ppn, pembetulan, ya, faktur
Topic 1: npwp, nya, ya, pph, pajak, terima, potong
Topic 2: kalo, kp, ya, orang, nilai, nya, kasih
Topic 3: pt, kalo, pajak, a, ppn, b, pph
Topic 4: kalo, password, wajahtersenyumlebardanmatatertutup, aktivasi, ya, pajak, kode
Topic 5: jasa, ya, peru